# Tutorial: Biomarker Selection for Prognosis Prediction

## Overview

This tutorial demonstrates how to use the Virtual Lab platform to identify prognostic biomarker genes from gene expression data. We will use the TCGA triple-negative breast cancer (TNBC) dataset as an example.

## Objectives

1. Load and explore gene expression data with survival outcomes
2. Apply multiple statistical methods for biomarker selection:
   - Cox Proportional Hazards regression
   - Log-rank test (Kaplan-Meier analysis)
   - Differential expression analysis
   - Elastic Net Cox regression
3. Identify consensus biomarkers across methods
4. Visualize and interpret results

## Dataset

**File:** `Example_TCGA_TNBC_data.csv`

- **Samples:** 144 TCGA triple-negative breast cancer patients
- **Features:** ~20,000 gene expression values (log-transformed)
- **Outcome variables:**
  - `OS`: Overall survival event (0 = censored, 1 = event)
  - `OS.year`: Overall survival time in years

---

## Step 1: Environment Setup

First, let's import the required libraries and set up our environment.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Statistical analysis
from scipy import stats
from lifelines import CoxPHFitter, KaplanMeierFitter
from lifelines.statistics import logrank_test

# Survival analysis
from sksurv.linear_model import CoxnetSurvivalAnalysis
from sksurv.util import Surv
from sklearn.preprocessing import StandardScaler

# Settings
import warnings
warnings.filterwarnings('ignore')

sns.set_style('whitegrid')
plt.rcParams['figure.dpi'] = 100

print("Libraries imported successfully!")

## Step 2: Load and Explore Data

Let's load the TCGA TNBC dataset and examine its structure.

In [ ]:
# Load data
data_path = "../Example_TCGA_TNBC_data.csv"
df = pd.read_csv(data_path)

# Remove quotes from column names if present
df.columns = df.columns.str.replace('"', '')

print(f"Dataset shape: {df.shape}")
print(f"\nFirst few rows:")
df.head()

In [ ]:
# Get gene names (exclude metadata columns)
gene_names = [col for col in df.columns if col not in ['sample', 'OS', 'OS.year']]

print(f"Number of samples: {len(df)}")
print(f"Number of genes: {len(gene_names)}")
print(f"\nOutcome distribution:")
print(f"  Events (death): {df['OS'].sum()}")
print(f"  Censored: {(1-df['OS']).sum()}")
print(f"\nSurvival time statistics (years):")
print(df['OS.year'].describe())

### Visualize Survival Data

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Overall survival distribution
ax = axes[0]
df['OS'].value_counts().plot(kind='bar', ax=ax, color=['lightblue', 'salmon'])
ax.set_xlabel('Outcome', fontsize=12)
ax.set_ylabel('Count', fontsize=12)
ax.set_title('Overall Survival Outcome Distribution', fontsize=13, fontweight='bold')
ax.set_xticklabels(['Censored', 'Event'], rotation=0)

# Survival time distribution
ax = axes[1]
ax.hist(df['OS.year'], bins=30, edgecolor='black', alpha=0.7)
ax.set_xlabel('Survival Time (years)', fontsize=12)
ax.set_ylabel('Frequency', fontsize=12)
ax.set_title('Survival Time Distribution', fontsize=13, fontweight='bold')

plt.tight_layout()
plt.show()

### Kaplan-Meier Curve for Overall Cohort

In [ ]:
from lifelines import KaplanMeierFitter

kmf = KaplanMeierFitter()
kmf.fit(df['OS.year'], df['OS'], label='Overall cohort')

fig, ax = plt.subplots(figsize=(10, 6))
kmf.plot_survival_function(ax=ax, ci_show=True)

ax.set_xlabel('Time (years)', fontsize=12)
ax.set_ylabel('Survival Probability', fontsize=12)
ax.set_title('Kaplan-Meier Survival Curve: TCGA TNBC Cohort', fontsize=14, fontweight='bold')
ax.grid(alpha=0.3)

# Add median survival time
median_survival = kmf.median_survival_time_
ax.axhline(0.5, color='red', linestyle='--', alpha=0.5, label=f'Median survival: {median_survival:.2f} years')
ax.legend()

plt.tight_layout()
plt.show()

print(f"Median survival time: {median_survival:.2f} years")

---

## Step 3: Method 1 - Cox Proportional Hazards Regression

Cox regression models the relationship between gene expression and survival time, accounting for censored data.

**Hazard Ratio (HR):**
- HR > 1: Higher expression associated with worse survival (risk factor)
- HR < 1: Higher expression associated with better survival (protective factor)

Let's perform univariate Cox regression for a few example genes.

In [ ]:
def perform_cox_regression(df, gene):
    """
    Perform Cox regression for a single gene
    """
    cox_df = pd.DataFrame({
        'time': df['OS.year'],
        'event': df['OS'],
        'gene_expr': df[gene]
    })
    
    cph = CoxPHFitter()
    cph.fit(cox_df, duration_col='time', event_col='event')
    
    return cph

# Example with a few genes
example_genes = gene_names[:5]

cox_results = []
for gene in example_genes:
    cph = perform_cox_regression(df, gene)
    
    cox_results.append({
        'gene': gene,
        'coef': cph.params_['gene_expr'],
        'HR': np.exp(cph.params_['gene_expr']),
        'p_value': cph.summary['p']['gene_expr'],
        'CI_lower': np.exp(cph.confidence_intervals_['gene_expr']['95% lower-bound']),
        'CI_upper': np.exp(cph.confidence_intervals_['gene_expr']['95% upper-bound'])
    })

cox_results_df = pd.DataFrame(cox_results)
print("Cox Regression Results (Example Genes):")
print(cox_results_df.to_string(index=False))

---

## Step 4: Method 2 - Log-rank Test (Kaplan-Meier Analysis)

The log-rank test compares survival curves between groups (e.g., high vs. low expression).

Let's demonstrate with an example gene.

In [ ]:
def plot_km_by_expression(df, gene):
    """
    Plot Kaplan-Meier curves for high vs low expression groups
    """
    # Split by median
    median_expr = df[gene].median()
    high_expr = df[gene] > median_expr
    
    # Fit KM curves
    kmf_high = KaplanMeierFitter()
    kmf_low = KaplanMeierFitter()
    
    kmf_high.fit(df.loc[high_expr, 'OS.year'], df.loc[high_expr, 'OS'], label='High expression')
    kmf_low.fit(df.loc[~high_expr, 'OS.year'], df.loc[~high_expr, 'OS'], label='Low expression')
    
    # Perform log-rank test
    result = logrank_test(
        df.loc[high_expr, 'OS.year'],
        df.loc[~high_expr, 'OS.year'],
        df.loc[high_expr, 'OS'],
        df.loc[~high_expr, 'OS']
    )
    
    # Plot
    fig, ax = plt.subplots(figsize=(10, 6))
    kmf_high.plot_survival_function(ax=ax, ci_show=True)
    kmf_low.plot_survival_function(ax=ax, ci_show=True)
    
    ax.set_xlabel('Time (years)', fontsize=12)
    ax.set_ylabel('Survival Probability', fontsize=12)
    ax.set_title(f'Kaplan-Meier Curve: {gene}\np-value = {result.p_value:.4f}',
                fontsize=14, fontweight='bold')
    ax.legend()
    ax.grid(alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    return result

# Example with first gene
example_gene = gene_names[0]
result = plot_km_by_expression(df, example_gene)

print(f"\nLog-rank test for {example_gene}:")
print(f"  Test statistic: {result.test_statistic:.4f}")
print(f"  P-value: {result.p_value:.4f}")

---

## Step 5: Method 3 - Differential Expression Analysis

Compare gene expression between patients with events vs. censored patients.

We'll use the Mann-Whitney U test (non-parametric).

In [ ]:
def differential_expression_analysis(df, gene):
    """
    Perform differential expression analysis for a single gene
    """
    event_mask = df['OS'] == 1
    
    expr_event = df.loc[event_mask, gene]
    expr_censored = df.loc[~event_mask, gene]
    
    # Mann-Whitney U test
    statistic, p_value = stats.mannwhitneyu(expr_event, expr_censored, alternative='two-sided')
    
    # Calculate fold change
    mean_event = expr_event.mean()
    mean_censored = expr_censored.mean()
    log_fc = mean_event - mean_censored
    
    return {
        'gene': gene,
        'mean_event': mean_event,
        'mean_censored': mean_censored,
        'log_FC': log_fc,
        'p_value': p_value
    }

# Example with a few genes
de_results = [differential_expression_analysis(df, gene) for gene in example_genes]
de_results_df = pd.DataFrame(de_results)

print("Differential Expression Results (Example Genes):")
print(de_results_df.to_string(index=False))

In [ ]:
# Visualize differential expression for an example gene
example_gene = gene_names[0]
event_mask = df['OS'] == 1

fig, ax = plt.subplots(figsize=(8, 6))

data_to_plot = [
    df.loc[~event_mask, example_gene],
    df.loc[event_mask, example_gene]
]

bp = ax.boxplot(data_to_plot, labels=['Censored', 'Event'],
                patch_artist=True, showmeans=True)

bp['boxes'][0].set_facecolor('lightblue')
bp['boxes'][1].set_facecolor('salmon')

ax.set_ylabel('Expression (log scale)', fontsize=12)
ax.set_xlabel('Outcome', fontsize=12)
ax.set_title(f'Expression Distribution: {example_gene}', fontsize=14, fontweight='bold')
ax.grid(alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

---

## Step 6: Run Complete Analysis with All Methods

Now let's use the comprehensive marker gene selection script to analyze all genes.

### Option 1: Run from Python

In [ ]:
# Import the MarkerGeneSelector class
import sys
sys.path.append('../biomarker_analysis/scripts')

from select_marker_genes import MarkerGeneSelector, Args

# Configure analysis
args = Args(
    input_file="../Example_TCGA_TNBC_data.csv",
    output_dir="../biomarker_results",
    n_top_genes=50,
    p_value_threshold=0.05,
    methods=["cox", "logrank", "differential", "elasticnet"],
    visualization=True,
    random_seed=42
)

# Initialize and run analysis
selector = MarkerGeneSelector(args)
selector.load_data()

# Run all methods
print("\n" + "="*80)
print("Running comprehensive biomarker analysis...")
print("="*80)

selector.method_cox_regression()
selector.method_logrank_test()
selector.method_differential_expression()
selector.method_elasticnet_cox()

# Get consensus genes
consensus = selector.get_consensus_genes()

print("\n=== Top 20 Consensus Genes ===")
print(consensus.head(20))

### Option 2: Run from Command Line

You can also run the analysis from the command line:

```bash
cd biomarker_analysis/scripts

python select_marker_genes.py \
    --input_file ../../Example_TCGA_TNBC_data.csv \
    --output_dir ../../biomarker_results \
    --n_top_genes 50 \
    --p_value_threshold 0.05 \
    --methods cox logrank differential elasticnet \
    --visualization
```

---

## Step 7: Explore Results

Let's examine the results from different methods.

In [ ]:
# Save results
selector.save_results(args.output_dir)
selector.visualize_results(args.output_dir)

print(f"\nResults saved to: {args.output_dir}/")
print("\nGenerated files:")
print("  - cox_results.csv")
print("  - logrank_results.csv")
print("  - differential_results.csv")
print("  - elasticnet_results.csv")
print("  - consensus_genes.csv")
print("  - top_genes_all_methods.csv")
print("  - marker_gene_analysis.pdf")

### Load and Compare Results

In [ ]:
# Load consensus genes
consensus_df = pd.read_csv(f"{args.output_dir}/consensus_genes.csv")

print("=== Consensus Genes Summary ===")
print(f"\nGenes appearing in multiple methods:")
for n_methods in sorted(consensus_df['n_methods'].unique(), reverse=True):
    count = (consensus_df['n_methods'] == n_methods).sum()
    print(f"  {n_methods} methods: {count} genes")

print("\n=== Top 30 Consensus Genes ===")
print(consensus_df.head(30).to_string(index=False))

In [ ]:
# Visualize method overlap
fig, ax = plt.subplots(figsize=(10, 6))

method_counts = consensus_df['n_methods'].value_counts().sort_index(ascending=False)

bars = ax.bar(method_counts.index, method_counts.values,
             color=sns.color_palette('viridis', len(method_counts)),
             edgecolor='black', linewidth=1.5)

# Add value labels
for bar in bars:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
           f'{int(height)}',
           ha='center', va='bottom', fontsize=12, fontweight='bold')

ax.set_xlabel('Number of Methods', fontsize=13)
ax.set_ylabel('Number of Genes', fontsize=13)
ax.set_title('Consensus Biomarkers Across Methods', fontsize=15, fontweight='bold')
ax.set_xticks(method_counts.index)
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

---

## Step 8: Detailed Analysis of Top Biomarkers

Let's examine the top consensus biomarkers in detail.

In [ ]:
# Get top consensus genes (appearing in all 4 methods)
top_consensus = consensus_df[consensus_df['n_methods'] == consensus_df['n_methods'].max()]

print(f"Genes appearing in {top_consensus['n_methods'].iloc[0]} methods:")
print(top_consensus.to_string(index=False))

# If no genes appear in all methods, get top genes by number of methods
if len(top_consensus) == 0:
    top_consensus = consensus_df.head(10)
    print(f"\nTop 10 consensus genes:")
    print(top_consensus.to_string(index=False))

In [ ]:
# Detailed analysis for top 3 genes
top_genes_to_analyze = consensus_df.head(3)['gene'].tolist()

for gene in top_genes_to_analyze:
    print("\n" + "="*80)
    print(f"Detailed Analysis: {gene}")
    print("="*80)
    
    # Get results from each method
    cox_result = selector.results['cox'][selector.results['cox']['gene'] == gene].iloc[0]
    logrank_result = selector.results['logrank'][selector.results['logrank']['gene'] == gene].iloc[0]
    de_result = selector.results['differential'][selector.results['differential']['gene'] == gene].iloc[0]
    
    print(f"\nCox Regression:")
    print(f"  Hazard Ratio: {cox_result['HR']:.3f} (95% CI: {cox_result['HR_95CI_lower']:.3f}-{cox_result['HR_95CI_upper']:.3f})")
    print(f"  P-value: {cox_result['p_value']:.2e}")
    
    print(f"\nLog-rank Test:")
    print(f"  P-value: {logrank_result['p_value']:.2e}")
    
    print(f"\nDifferential Expression:")
    print(f"  Log FC (Event vs Censored): {de_result['log_FC']:.3f}")
    print(f"  P-value: {de_result['p_value']:.2e}")
    
    # Plot KM curve
    plot_km_by_expression(df, gene)

---

## Step 9: Export Results for Further Analysis

Create a summary report of the top biomarkers.

In [ ]:
# Create comprehensive summary for top genes
top_n = 20
top_genes = consensus_df.head(top_n)['gene'].tolist()

summary_data = []

for gene in top_genes:
    cox_res = selector.results['cox'][selector.results['cox']['gene'] == gene].iloc[0]
    logrank_res = selector.results['logrank'][selector.results['logrank']['gene'] == gene].iloc[0]
    de_res = selector.results['differential'][selector.results['differential']['gene'] == gene].iloc[0]
    
    summary_data.append({
        'Gene': gene,
        'N_Methods': consensus_df[consensus_df['gene'] == gene]['n_methods'].iloc[0],
        'Methods': consensus_df[consensus_df['gene'] == gene]['methods'].iloc[0],
        'Cox_HR': f"{cox_res['HR']:.3f}",
        'Cox_pval': f"{cox_res['p_value']:.2e}",
        'Logrank_pval': f"{logrank_res['p_value']:.2e}",
        'DE_logFC': f"{de_res['log_FC']:.3f}",
        'DE_pval': f"{de_res['p_value']:.2e}"
    })

summary_df = pd.DataFrame(summary_data)

print("=== Top 20 Biomarkers Summary ===")
print(summary_df.to_string(index=False))

# Save summary
summary_path = f"{args.output_dir}/top_biomarkers_summary.csv"
summary_df.to_csv(summary_path, index=False)
print(f"\nSummary saved to: {summary_path}")

---

## Conclusion

### Summary

In this tutorial, we demonstrated how to:

1. **Load and explore** survival data with gene expression profiles
2. **Apply multiple methods** for biomarker discovery:
   - Cox Proportional Hazards regression (hazard ratios)
   - Log-rank test (survival curve comparison)
   - Differential expression analysis (event vs. censored)
   - Elastic Net Cox regression (regularized feature selection)
3. **Identify consensus biomarkers** that are significant across multiple methods
4. **Visualize and interpret** results with comprehensive plots

### Next Steps

1. **Validate biomarkers** in independent datasets
2. **Build predictive models** using selected genes
3. **Perform pathway analysis** to understand biological mechanisms
4. **Literature review** of identified biomarkers
5. **Experimental validation** in laboratory settings

### Key Files Generated

- `consensus_genes.csv`: Genes ranked by appearance across methods
- `{method}_results.csv`: Detailed results for each method
- `top_biomarkers_summary.csv`: Comprehensive summary of top genes
- `marker_gene_analysis.pdf`: Visualization plots

---

## References

- Cox, D. R. (1972). Regression models and life-tables. *Journal of the Royal Statistical Society*.
- Kaplan, E. L., & Meier, P. (1958). Nonparametric estimation from incomplete observations. *Journal of the American Statistical Association*.
- Simon, N., et al. (2011). Regularization paths for Cox's proportional hazards model via coordinate descent. *Journal of Statistical Software*.
- TCGA Network (2012). Comprehensive molecular portraits of human breast tumours. *Nature*.

---

**For questions or issues, please refer to the Virtual Lab documentation or create an issue on GitHub.**